In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from a500_utils.helpersII import make_tuple
from thermlib import thermfuncsII as tf
import pandas as pd
with pd.HDFStore('dumpvapor.h5') as f:
    df_result=f['profile']
out=df_result.plot('time','h')

In [ ]:
#
# capture the last timestep and convert to a named tuple
#

out=df_result.iloc[-1]
steady_state=make_tuple(out.to_dict())
steady_state

### make a buoyancy flux profile

In [ ]:
def flux_prof(z,coeffs):
    Fout=coeffs.Fsfc + (coeffs.Finv - coeffs.Fsfc)*(z/coeffs.zinv)
    return Fout

zvals=np.linspace(0,steady_state.h,30)
coeffs_thetal=make_tuple(dict(Fsfc=steady_state.T_flux_0,Finv=steady_state.entflux_theta,
                       zinv=steady_state.h))
coeffs_qt=make_tuple(dict(Fsfc=steady_state.q_flux_0,Finv=steady_state.entflux_qv,
                       zinv=steady_state.h))
flux_thetal=flux_prof(zvals,coeffs_thetal)
flux_qt=flux_prof(zvals,coeffs_qt)
df_flux=pd.DataFrame.from_records(zip(zvals,flux_thetal,flux_qt),columns=['z','flux_thetal','flux_qt'])
fig,ax = plt.subplots(1,2,figsize=(12,10))
df_flux.plot('flux_thetal','z',ax=ax[0])
out=ax[1].plot(flux_qt*2.5e6,zvals)

In [ ]:
thetal = steady_state.theta
qt = steady_state.qv
buoyancy=[]
for z,thetal_flux,qt_flux in zip(zvals,flux_thetal,flux_qt):
    #assume 10 kPa = 1000 meters
    press = 100. - (z/100.)  #kPa
    A,B,issat = tf.calc_ABcoeffs(thetal,qt,press)
    buoyancy.append(A*thetal_flux + B*qt_flux)
g=9.8
buoy_flux =np.array(buoyancy)
out=plt.plot(buoy_flux,zvals)
wstar3 = np.sum(buoy_flux[1:]*np.diff(zvals))*2.5*g/thetal
print('wstar= {:5.3f} m/s'.format(wstar3**0.333))